# Aspects (plus) avancés pour la bioinformatique: recherche dans une base de données distante, version XML et NCBI Entrez
Le processus de collecte d'informations pour un processus d'annotation requiert des recherches sur des bases de données pré-existantes. De manière non-programmatique, on commence par aller sur une page web, on inscrit un ou plusieurs critères de recherche et le site nous retourne un ou des pages de résultats. Ok, c'est cool mais comment intégrer ça dans une structure de données? Il est tout è fait possible de le faire programmatiquement via Python. Dans les exemples qui suivent, nous allons utiliser les services E-Utils du NCBI pour faire notre recherche et notre collecte d'informations.

Le NCBI (National Center for Biotechnology Information) est le "uber" site de stockage des données biologiques et chimiques (PubChem); le NCBI offre gratuitement un océan d'informations qu'on peut consulter pour créer des annotations selon nos besoins, accessibles via la suite d'outils EUtils dans un URL.

Dans l'exemple qui suit, on recherchera un ensemble de variations génétiques en utilisant la base de données dbVar du NCBI pour notre ami, le gène CTTN. La logique de recherche des infos reste la même que pour l'uuilisation de MyGene.info:

 - On recherche un ensemble de données (ici, les variations génvtiques) se retrouvant associées au gène CTTN en utilisant l'utilisaire ESearch;
 - Pour chaque variation retrouvée, on veux avoir les informations disponibles sur cette variation en utilisant l'utilitaire EFetch.

Pour transformer "instantanément" un flux xml en dictionnaire, il nous faudra utiliser le module xmltodict.

### Installation des modules nécessaires

Avant de commencer, assurons-nous uqe les modules nécessaires soient installés sur notre plateforme avec pip:

In [2]:
!pip install xmltodict

  Using cached xmltodict-0.13.0-py2.py3-none-any.whl (10.0 kB)


In [2]:
import requests
import xmltodict
# Pas necassaire pour le travail à faire mais
# simplement pour afficher ce qui revient du serveur
import xml.dom.minidom

#
# URl statique: correspond à la portion invariable de l'URL
#
search_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?"
#
# Preparation de la requete
#
headers = {'content-type': 'application/x-www-form-urlencoded'}
#
# Création des paramètres. 
# Notez que ça pourrait etre fait de maniere programmatique 
#
# Je limite mon résultat aux deux premiers SNP
#
params = {
    "db" : "snp",
    "term": "CTTN",
    "retMax": 2,
    "rettype": "XML"
}
#
# Envoyez la requete
#
res = requests.get(search_url, params=params,headers=headers)
#
# Retirez les commentaires pour voir..
# Juste pour vous montrer de quoi à l'air le format XML :-)
#
#print("=========== Sortie XML ===========\n")
#temp = xml.dom.minidom.parseString(res.content) 
#new_xml = temp.toprettyxml(indent=" ") 
#print(new_xml) 
#print("==================================\n")

# C'est avec ça que nous allons travailler!
aDict = xmltodict.parse(res.content)
print("Voici le contenu de notre dictionnaire:")
print(aDict)
# On recupère le contenu de la clé eSearchResult
# créé par le parsing du contenu XML
aDict = aDict["eSearchResult"]

#
# Maintenant on récupère les infos de chacun des éléments Id sous IdList
#
idList = aDict["IdList"]
idList = idList["Id"]

for i in idList:
    fetch_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=snp&id="+i+"&retmode=xml"
    res = requests.get(fetch_url,headers=headers)
    aVar = xmltodict.parse(res.content)
    # Imprimons l'identificateur rs
    print("rs"+i)
    myDict = aVar["ExchangeSet"]
    print(myDict["DocumentSummary"]["CHRPOS"])
    print(myDict["DocumentSummary"]["DOCSUM"])

{'eSearchResult': {'Count': '17007', 'RetMax': '2', 'RetStart': '0', 'IdList': {'Id': ['2135607633', '2135607342']}, 'TranslationSet': None, 'TranslationStack': {'TermSet': {'Term': 'CTTN[All Fields]', 'Field': 'All Fields', 'Count': '17007', 'Explode': 'N'}, 'OP': 'GROUP'}, 'QueryTranslation': 'CTTN[All Fields]'}}
rs2135607633
11:70436930
HGVS=NC_000011.10:g.70436930T>G,NC_000011.9:g.70283036T>G,NG_029881.1:g.43425T>G,NW_021160005.1:g.198218T>G|SEQ=[T/G]|LEN=1|GENE=CTTN:2017
rs2135607342
11:70436802
HGVS=NC_000011.10:g.70436802A>C,NC_000011.9:g.70282908A>C,NG_029881.1:g.43297A>C,NW_021160005.1:g.198090A>C|SEQ=[A/C]|LEN=1|GENE=CTTN:2017
